In [1]:
%display latex

# 6-dimensional CSLA - NB 10 - Section 7

In this notebook we study the relation between the CSLA which admit metrics with nontrivial index of symmetry and the nilsoliton metrics. In particular we prove that every nilsoliton metric has nontrivial index of symmetry. This is explained in Section 7 of the article, but we recall some definitions and results for the benefit of the reader.

Let $\mathfrak h$ be a CSLA of dimension $6$. Recall that a left-invariant metric $g$ on $\mathfrak h$ is called a *nilsoliton* if
$$
\operatorname{Ric}_g = c \, I_6 + D \qquad (*)
$$
for some $c \in \mathbb R$ and some derivation $D$. As usual, we identify $g$ and $D$ by their matrix representation in the standard basis $e_1, \ldots, e_6$. Since the Ricci operator is symmetric,  $D$ is in diagonal form with respect to the standard basis. We will asumme that $g$ is also in diagonal form in this basis. A nilsoliton metric is unique up to scaling, hence we do not lose generality by making this assumption (this is seen a posteriori by finding the solution of (*) among the diagonal metrics). 

For computing the Ricci operator we use the well-known formula
$$
\operatorname{Ric}_g(\tilde e_j, \tilde e_h) = \frac12\sum_{i,k} c_{iki} (c_{kjh} + c_{khj}) + \frac12 c_{ikh} c_{ikj} - c_{ijk} c_{khi} + c_{iki} c_{jhk} - c_{ijk} c_{ihk} 
$$
where $\tilde e_1, \ldots, \tilde e_6$ is an orthonormal basis and
$$
c_{ijk} = g([\tilde e_i, \tilde e_j], \tilde e_k ).
$$

The following function `Ric_on` computes the Ricci operator with respect to an orthonormal basis. Notice that if $\tilde e_1, \ldots, \tilde e_6$ is obtained from $e_1, \ldots, e_6$ by the Gram-Schmidt process, then the matrices of the Ricci operator in both basis coincide.

In [4]:
def inner_product_from_matrix(hh, g):
    return lambda x, y: x.to_vector() * g * y.to_vector()

def gram_schmidt(hh, inner_product):
    hh_on_basis = []
    for i, v in enumerate(hh.basis().values()):
        u = v
        for j in range(i):
            u -= inner_product(v, hh_on_basis[j]) * hh_on_basis[j]
        hh_on_basis.append( hh( ((1 / sqrt(inner_product(u, u))) * u).to_vector().canonicalize_radical() ))
    return hh_on_basis

def bracket_contracted(hh, inner_product, on_basis):
    bc = {(i,j,k): None for i in range(6) for j in range(6) for k in range(6)}
    for index in bc.keys():
        i, j, k = index
        bc[index] = inner_product(on_basis[i].bracket(on_basis[j]), on_basis[k])
    return bc

def Ric_ob(hh, bc, j, h):
    hh_dim = hh.dimension()
    return sum(sum( bc[(i,k,i)] * (bc[(k,j,h)] + bc[(k,h,j)]) / 2 + bc[(i,k,h)] * bc[(i,k,j)] / 4 - bc[(i,j,k)] * bc[(k,h,i)] / 2 + bc[(i,k,i)] * bc[(j,h,k)] / 2 - bc[(i,j,k)] * bc[(i,h,k)] / 2 for i in range(hh_dim)) for k in range(hh_dim))

def Ric_on(hh, g, on_basis):
    ip = inner_product_from_matrix(hh, g)
    on_basis = gram_schmidt(hh, ip)
    bc = bracket_contracted(hh, ip, on_basis)
    return matrix(6, 6, lambda i, j: Ric_ob(hh, bc, i, j))

Next we compute the nilsoliton metrics for the Lie algebras which admit left-invariant metrics with nontrivial index of symmetry. We also give explicit isometric isomorphisms to the nilsoliton metrics classified by C. Will in 2011.

In [5]:
load("CSLA.sage")

## Case $\mathfrak h_9$

In [6]:
construct_the_algebra("h9_tris")
%time generic_metric(alg_name) # long time
g, Sigma

CPU times: user 6.98 s, sys: 937 ms, total: 7.92 s
Wall time: 7.05 s


(
[          1           0           0           0           0           0]
[          0           1           0           0           0           0]
[          0           0 s0^2 + s1^2       s1*s2       s1*s3           0]
[          0           0       s1*s2    s2^2 + 1       s2*s3           0]
[          0           0       s1*s3       s2*s3 s3^2 + s4^2       s4*s5]
[          0           0           0           0       s4*s5        s5^2],

[ 1  0  0  0  0  0]
[ 0  1  0  0  0  0]
[ 0  0 s0  0  0  0]
[ 0  0  0  1  0  0]
[ 0  0 s1 s2 s3  0]
[ 0  0  0  0 s4 s5]
)

We can avoid very long computations by working with diagonal metrics. 

In [7]:
g_diag = g.subs(s1=0,s2=0,s4=0)
Sigma_diag = Sigma.subs(s1=0,s2=0,s4=0)
g_diag

[   1    0    0    0    0    0]
[   0    1    0    0    0    0]
[   0    0 s0^2    0    0    0]
[   0    0    0    1    0    0]
[   0    0    0    0 s3^2    0]
[   0    0    0    0    0 s5^2]

Now we compute the Ricci tensor.

In [8]:
ip = inner_product_from_matrix(alg, g_diag)
on_basis = gram_schmidt(alg, ip)
display(on_basis)
Ric_on_basis = Ric_on(alg, g_diag, on_basis)
Ric_on_basis

[e1, e2, 1/s0*e3, e4, 1/s3*e5, 1/s5*e6]

[    -1/2*s3^2 - 1/2*s5^2/s3^2                             0                             0                             0                             0                             0]
[                            0     -1/2*s3^2 - 1/2*s5^2/s0^2                             0                             0                             0                             0]
[                            0                             0                -1/2*s5^2/s0^2                             0                             0                             0]
[                            0                             0                             0                             0                             0                             0]
[                            0                             0                             0                             0      1/2*s3^2 - 1/2*s5^2/s3^2                             0]
[                            0                             0                             0                             0                             0 1/2*s5^2/s0^2 + 1/2*s5^2/s3^2]

Now we solve equation $(*)$. Recall that the derivation $D$ can be assumed in diagonal form.

In [9]:
c = var("c")
D = der_gen_diag
display(D)
nilsoliton_eqs = (Ric_on_basis - c * identity_matrix(alg.dimension()) - D).coefficients()
nilsoliton_eqs

[       d0         0         0         0         0         0]
[        0        d2         0         0         0         0]
[        0         0      2*d0         0         0         0]
[        0         0         0        d8         0         0]
[        0         0         0         0   d0 + d2         0]
[        0         0         0         0         0 2*d0 + d2]

[-1/2*s3^2 - c - d0 - 1/2*s5^2/s3^2,
 -1/2*s3^2 - c - d2 - 1/2*s5^2/s0^2,
 -c - 2*d0 - 1/2*s5^2/s0^2,
 -c - d8,
 1/2*s3^2 - c - d0 - d2 - 1/2*s5^2/s3^2,
 -c - 2*d0 - d2 + 1/2*s5^2/s0^2 + 1/2*s5^2/s3^2]

In [10]:
g_diag_vars = list(set(Sigma.diagonal()) - {1})
incg = [c] + der_vars_diag + g_diag_vars
display(incg)
nilsoliton_sols = solve(nilsoliton_eqs, incg, algorithm='sympy')
for sol in nilsoliton_sols:
    display(sol)

[c, d0, d2, d8, s5, s0, s3]

{c: -7/4*sqrt(s5^2),
 d0: 3/4*sqrt(s5^2),
 d2: sqrt(s5^2),
 d8: 7/4*sqrt(s5^2),
 s0: -sqrt(2)*(s5^2)^(1/4),
 s3: -(s5^2)^(1/4)}

{c: -7/4*sqrt(s5^2),
 d0: 3/4*sqrt(s5^2),
 d2: sqrt(s5^2),
 d8: 7/4*sqrt(s5^2),
 s0: -sqrt(2)*(s5^2)^(1/4),
 s3: (s5^2)^(1/4)}

{c: -7/4*sqrt(s5^2),
 d0: 3/4*sqrt(s5^2),
 d2: sqrt(s5^2),
 d8: 7/4*sqrt(s5^2),
 s0: sqrt(2)*(s5^2)^(1/4),
 s3: -(s5^2)^(1/4)}

{c: -7/4*sqrt(s5^2),
 d0: 3/4*sqrt(s5^2),
 d2: sqrt(s5^2),
 d8: 7/4*sqrt(s5^2),
 s0: sqrt(2)*(s5^2)^(1/4),
 s3: (s5^2)^(1/4)}

{c: 7/4*sqrt(s5^2),
 d0: -3/4*sqrt(s5^2),
 d2: -sqrt(s5^2),
 d8: -7/4*sqrt(s5^2),
 s0: -sqrt(2)*sqrt(-sqrt(s5^2)),
 s3: -I*(s5^2)^(1/4)}

{c: 7/4*sqrt(s5^2),
 d0: -3/4*sqrt(s5^2),
 d2: -sqrt(s5^2),
 d8: -7/4*sqrt(s5^2),
 s0: -sqrt(2)*sqrt(-sqrt(s5^2)),
 s3: I*(s5^2)^(1/4)}

{c: 7/4*sqrt(s5^2),
 d0: -3/4*sqrt(s5^2),
 d2: -sqrt(s5^2),
 d8: -7/4*sqrt(s5^2),
 s0: sqrt(2)*sqrt(-sqrt(s5^2)),
 s3: -I*(s5^2)^(1/4)}

{c: 7/4*sqrt(s5^2),
 d0: -3/4*sqrt(s5^2),
 d2: -sqrt(s5^2),
 d8: -7/4*sqrt(s5^2),
 s0: sqrt(2)*sqrt(-sqrt(s5^2)),
 s3: I*(s5^2)^(1/4)}

Since $g = g_\sigma$ with $\sigma \in \Sigma \subset \rm T_6^+$ the only meaningful solution is the fourth one. Warning: the place of the meaningful solution in `nilsoliton_sols` may vary.

In [11]:
g_nil = g_diag.subs(nilsoliton_sols[3]).canonicalize_radical()
Sigma_nil = Sigma_diag.subs(nilsoliton_sols[3]).canonicalize_radical()
ip_nil = inner_product_from_matrix(alg, g_nil)
g_nil

[   1    0    0    0    0    0]
[   0    1    0    0    0    0]
[   0    0 2*s5    0    0    0]
[   0    0    0    1    0    0]
[   0    0    0    0   s5    0]
[   0    0    0    0    0 s5^2]

It follows from Theorem 6.2 that the nilsoliton metrics have nontrivial index of symmetry.

In [12]:
Sigma, Sigma_nil

(
[ 1  0  0  0  0  0]  [               1                0                0                0                0                0]
[ 0  1  0  0  0  0]  [               0                1                0                0                0                0]
[ 0  0 s0  0  0  0]  [               0                0 sqrt(2)*sqrt(s5)                0                0                0]
[ 0  0  0  1  0  0]  [               0                0                0                1                0                0]
[ 0  0 s1 s2 s3  0]  [               0                0                0                0         sqrt(s5)                0]
[ 0  0  0  0 s4 s5], [               0                0                0                0                0               s5]
)

### Isometry with the nilsoliton metric given in [Will11]

The nilsoliton metric from the 2011 paper of Will which is isomorphic to $\mathfrak h_9$ is the number 27 given by $(0, 0, 0, 0, 2^\frac12 \, \mathbf{12}, \mathbf{14} + 2^\frac12 \, \mathbf{25})$.

In [13]:
d_h = {('x1', 'x2'): {'x5': sqrt(2)}, ('x1', 'x4'): {'x6': 1}, ('x2', 'x5'): {'x6': sqrt(2)}} 
hh_will.<x1,x2,x3,x4,x5,x6> = LieAlgebra(SR, d_h, nilpotent=True) 
for kk in hh_will.structure_coefficients().keys():
    print(kk, " = ", hh_will.structure_coefficients()[kk])

('x1', 'x2')  =  sqrt(2)*x5
('x1', 'x4')  =  x6
('x2', 'x5')  =  sqrt(2)*x6


Notice that the basis $x_1, \ldots, x_6$ is orthonormal by definition. We propose the following isomorphism between `hh_will` and `alg`.

In [14]:
e1, e2, e3, e4, e5, e6 = alg.basis()
f1, f2, f3, f4, f5, f6 = e2, e1, e4, -1/2*e3, 1/2*sqrt(2)*e5, 1/2*e6
hh_will.morphism({x1:f1, x2:f2, x3:f3, x4:f4, x5:f5, x6:f6})

Lie algebra morphism:
  From: Nilpotent Lie algebra on 6 generators (x1, x2, x3, x4, x5, x6) over Symbolic Ring
  To:   Nilpotent Lie algebra on 6 generators (e1, e2, e3, e4, e5, e6) over Symbolic Ring
  Defn: x1 |--> e2
        x2 |--> e1
        x3 |--> e4
        x4 |--> -1/2*e3
        x5 |--> 1/2*sqrt(2)*e5
        x6 |--> 1/2*e6

And now we choose $s_5$ in order to make this isomorphism and isometry. 

In [15]:
on_basis_nil = [f1, f2, f3, f4, f5, f6]
matrix(6, 6, lambda i, j: ip_nil(on_basis_nil[i], on_basis_nil[j])).canonicalize_radical()

[       1        0        0        0        0        0]
[       0        1        0        0        0        0]
[       0        0        1        0        0        0]
[       0        0        0   1/2*s5        0        0]
[       0        0        0        0   1/2*s5        0]
[       0        0        0        0        0 1/4*s5^2]

So it follows that $s_5 = 2$.

In [16]:
g_will = g_nil.subs(s5=2)
g_will

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 4 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 2 0]
[0 0 0 0 0 4]

## Case $\mathfrak h_{10}$

In [17]:
construct_the_algebra("h10")
%time generic_metric(alg_name) # long time
g, Sigma

CPU times: user 7.53 s, sys: 78 ms, total: 7.61 s
Wall time: 6.77 s


(
[                 1                  0                  0                  0                  0                  0]
[                 0                  1                  0                  0                  0                  0]
[                 0                  0                  1                  0                  0                  0]
[                 0                  0                  0 s0^2 + s1^2 + s3^2      s1*s2 + s3*s4              s3*s5]
[                 0                  0                  0      s1*s2 + s3*s4        s2^2 + s4^2              s4*s5]
[                 0                  0                  0              s3*s5              s4*s5               s5^2],

[ 1  0  0  0  0  0]
[ 0  1  0  0  0  0]
[ 0  0  1  0  0  0]
[ 0  0  0 s0  0  0]
[ 0  0  0 s1 s2  0]
[ 0  0  0 s3 s4 s5]
)

We can avoid very long computations by working with diagonal metrics. 

In [18]:
g_diag = g.subs(s1=0,s3=0,s4=0)
Sigma_diag = Sigma.subs(s1=0,s3=0,s4=0)
g_diag

[   1    0    0    0    0    0]
[   0    1    0    0    0    0]
[   0    0    1    0    0    0]
[   0    0    0 s0^2    0    0]
[   0    0    0    0 s2^2    0]
[   0    0    0    0    0 s5^2]

Now we compute the Ricci tensor.

In [19]:
ip = inner_product_from_matrix(alg, g_diag)
on_basis = gram_schmidt(alg, ip)
display(on_basis)
Ric_on_basis = Ric_on(alg, g_diag, on_basis)
Ric_on_basis

[e1, e2, e3, 1/s0*e4, 1/s2*e5, 1/s5*e6]

[-1/2*s0^2 - 1/2*s2^2 - 1/2*s5^2/s0^2                                    0                                    0                                    0                                    0                                    0]
[                                   0                            -1/2*s0^2                                    0                                    0                                    0                                    0]
[                                   0                                    0                            -1/2*s2^2                                    0                                    0                                    0]
[                                   0                                    0                                    0             1/2*s0^2 - 1/2*s5^2/s0^2                                    0                                    0]
[                                   0                                    0                                    0                                    0                             1/2*s2^2                                    0]
[                                   0                                    0                                    0                                    0                                    0                        1/2*s5^2/s0^2]

Now we solve equation $(*)$. Recall that the derivation $D$ can be assumed in diagonal form

In [20]:
c = var("c")
D = der_gen_diag
display(D)
nilsoliton_eqs = (Ric_on_basis - c * identity_matrix(alg.dimension()) - D).coefficients()
nilsoliton_eqs

[       d0         0         0         0         0         0]
[        0        d2         0         0         0         0]
[        0         0        d5         0         0         0]
[        0         0         0   d0 + d2         0         0]
[        0         0         0         0   d0 + d5         0]
[        0         0         0         0         0 2*d0 + d2]

[-1/2*s0^2 - 1/2*s2^2 - c - d0 - 1/2*s5^2/s0^2,
 -1/2*s0^2 - c - d2,
 -1/2*s2^2 - c - d5,
 1/2*s0^2 - c - d0 - d2 - 1/2*s5^2/s0^2,
 1/2*s2^2 - c - d0 - d5,
 -c - 2*d0 - d2 + 1/2*s5^2/s0^2]

In [21]:
g_diag_vars = list(set(Sigma.diagonal()) - {1})
incg = [c] + der_vars_diag + g_diag_vars
display(incg)
nilsoliton_sols = solve(nilsoliton_eqs, incg, algorithm='sympy')
for sol in nilsoliton_sols:
    display(sol)

[c, d0, d2, d5, s5, s0, s2]

{c: -7/4*sqrt(s5^2),
 d0: 1/2*sqrt(s5^2),
 d2: 5/4*sqrt(s5^2),
 d5: 3/2*sqrt(s5^2),
 s0: -(s5^2)^(1/4),
 s2: -1/2*sqrt(2)*(s5^2)^(1/4)}

{c: -7/4*sqrt(s5^2),
 d0: 1/2*sqrt(s5^2),
 d2: 5/4*sqrt(s5^2),
 d5: 3/2*sqrt(s5^2),
 s0: -(s5^2)^(1/4),
 s2: 1/2*sqrt(2)*(s5^2)^(1/4)}

{c: -7/4*sqrt(s5^2),
 d0: 1/2*sqrt(s5^2),
 d2: 5/4*sqrt(s5^2),
 d5: 3/2*sqrt(s5^2),
 s0: (s5^2)^(1/4),
 s2: -1/2*sqrt(2)*(s5^2)^(1/4)}

{c: -7/4*sqrt(s5^2),
 d0: 1/2*sqrt(s5^2),
 d2: 5/4*sqrt(s5^2),
 d5: 3/2*sqrt(s5^2),
 s0: (s5^2)^(1/4),
 s2: 1/2*sqrt(2)*(s5^2)^(1/4)}

{c: 7/4*sqrt(s5^2),
 d0: -1/2*sqrt(s5^2),
 d2: -5/4*sqrt(s5^2),
 d5: -3/2*sqrt(s5^2),
 s0: -sqrt(-sqrt(s5^2)),
 s2: -1/2*I*sqrt(2)*(s5^2)^(1/4)}

{c: 7/4*sqrt(s5^2),
 d0: -1/2*sqrt(s5^2),
 d2: -5/4*sqrt(s5^2),
 d5: -3/2*sqrt(s5^2),
 s0: -sqrt(-sqrt(s5^2)),
 s2: 1/2*I*sqrt(2)*(s5^2)^(1/4)}

{c: 7/4*sqrt(s5^2),
 d0: -1/2*sqrt(s5^2),
 d2: -5/4*sqrt(s5^2),
 d5: -3/2*sqrt(s5^2),
 s0: sqrt(-sqrt(s5^2)),
 s2: -1/2*I*sqrt(2)*(s5^2)^(1/4)}

{c: 7/4*sqrt(s5^2),
 d0: -1/2*sqrt(s5^2),
 d2: -5/4*sqrt(s5^2),
 d5: -3/2*sqrt(s5^2),
 s0: sqrt(-sqrt(s5^2)),
 s2: 1/2*I*sqrt(2)*(s5^2)^(1/4)}

Since $g = g_\sigma$ with $\sigma \in \Sigma \subset \rm T_6^+$ the only meaningful solution is the fourth one. Warning: the place of the meaningful solution in `nilsoliton_sols` may vary.

In [19]:
g_nil = g_diag.subs(nilsoliton_sols[3]).canonicalize_radical()
Sigma_nil = Sigma_diag.subs(nilsoliton_sols[3]).canonicalize_radical()
ip_nil = inner_product_from_matrix(alg, g_nil)
g_nil

[     1      0      0      0      0      0]
[     0      1      0      0      0      0]
[     0      0      1      0      0      0]
[     0      0      0     s5      0      0]
[     0      0      0      0 1/2*s5      0]
[     0      0      0      0      0   s5^2]

It follows from Theorem 6.4 that the nilsoliton metrics have nontrivial index of symmetry.

In [22]:
Sigma, Sigma_nil

(
[ 1  0  0  0  0  0]  [               1                0                0                0                0                0]
[ 0  1  0  0  0  0]  [               0                1                0                0                0                0]
[ 0  0  1  0  0  0]  [               0                0 sqrt(2)*sqrt(s5)                0                0                0]
[ 0  0  0 s0  0  0]  [               0                0                0                1                0                0]
[ 0  0  0 s1 s2  0]  [               0                0                0                0         sqrt(s5)                0]
[ 0  0  0 s3 s4 s5], [               0                0                0                0                0               s5]
)

### Isometry with the nilsoliton metric given in [Will11]

The nilsoliton metric from the 2011 paper of Will which is isomorphic to $\mathfrak h_{10}$ is the number 23 given by $(0, 0, 0, 2^\frac12 \mathbf{12}, \mathbf{13}, 2^\frac12 \mathbf{14})$.

In [23]:
d_h = {('x1', 'x2'): {'x4': sqrt(2)}, ('x1', 'x3'): {'x5': 1}, ('x1', 'x4'): {'x6': sqrt(2)}} 
hh_will.<x1,x2,x3,x4,x5,x6> = LieAlgebra(SR, d_h, nilpotent=True) 
#hh_will_dim = hh_will.dimension()
for kk in hh_will.structure_coefficients().keys():
    print(kk, " = ", hh_will.structure_coefficients()[kk])

('x1', 'x2')  =  sqrt(2)*x4
('x1', 'x3')  =  x5
('x1', 'x4')  =  sqrt(2)*x6


Notice that the basis $x_1, \ldots, x_6$ is orthonormal by definition. We propose the following isomorphism between `hh_will` and `alg`.

In [24]:
e1, e2, e3, e4, e5, e6 = alg.basis()
f1, f2, f3, f4, f5, f6 = e1, e2, e3, -1/sqrt(2)*e4, -e5, 1/2*e6
hh_will.morphism({x1:f1, x2:f2, x3:f3, x4:f4, x5:f5, x6:f6})

Lie algebra morphism:
  From: Nilpotent Lie algebra on 6 generators (x1, x2, x3, x4, x5, x6) over Symbolic Ring
  To:   Nilpotent Lie algebra on 6 generators (e1, e2, e3, e4, e5, e6) over Symbolic Ring
  Defn: x1 |--> e1
        x2 |--> e2
        x3 |--> e3
        x4 |--> -1/2*sqrt(2)*e4
        x5 |--> -e5
        x6 |--> 1/2*e6

We choose $s_5$ in order to make this isomorphism and isometry. 

In [25]:
on_basis_nil = [f1, f2, f3, f4, f5, f6]
matrix(6, 6, lambda i, j: ip_nil(on_basis_nil[i], on_basis_nil[j])).canonicalize_radical()

[       1        0        0        0        0        0]
[       0        1        0        0        0        0]
[       0        0     2*s5        0        0        0]
[       0        0        0      1/2        0        0]
[       0        0        0        0       s5        0]
[       0        0        0        0        0 1/4*s5^2]

So it follows that $s_5 = 2$.

In [26]:
g_will = g_nil.subs(s5=2)
g_will

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 4 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 2 0]
[0 0 0 0 0 4]

## Case $\mathfrak h_{21}$

In [27]:
construct_the_algebra("h21")
%time generic_metric(alg_name) # long time
g, Sigma

CPU times: user 4.67 s, sys: 31 ms, total: 4.7 s
Wall time: 4.23 s


(
[                 1                  0                  0                  0                  0                  0]
[                 0                  1                  0                  0                  0                  0]
[                 0                  0    s0^2 + s2^2 + 1      s0*s1 + s2*s3              s2*s4                  0]
[                 0                  0      s0*s1 + s2*s3 s1^2 + s3^2 + s5^2      s3*s4 + s5*s6              s5*s7]
[                 0                  0              s2*s4      s3*s4 + s5*s6        s4^2 + s6^2              s6*s7]
[                 0                  0                  0              s5*s7              s6*s7               s7^2],

[ 1  0  0  0  0  0]
[ 0  1  0  0  0  0]
[ 0  0  1  0  0  0]
[ 0  0 s0 s1  0  0]
[ 0  0 s2 s3 s4  0]
[ 0  0  0 s5 s6 s7]
)

We can avoid very long computations by working with diagonal metrics. 

In [28]:
g_diag = g.subs(s0=0,s2=0,s3=0,s5=0,s6=0)
Sigma_diag = Sigma.subs(s0=0,s2=0,s3=0,s5=0,s6=0)
g_diag

[   1    0    0    0    0    0]
[   0    1    0    0    0    0]
[   0    0    1    0    0    0]
[   0    0    0 s1^2    0    0]
[   0    0    0    0 s4^2    0]
[   0    0    0    0    0 s7^2]

Now we compute the Ricci tensor.

In [29]:
ip = inner_product_from_matrix(alg, g_diag)
on_basis = gram_schmidt(alg, ip)
display(on_basis)
Ric_on_basis = Ric_on(alg, g_diag, on_basis)
Ric_on_basis

[e1, e2, e3, 1/s1*e4, 1/s4*e5, 1/s7*e6]

[-1/2*s1^2 - 1/2*s4^2/s1^2 - 1/2*s7^2/s4^2                                         0                                         0                                         0                                         0                                         0]
[                                        0                                 -1/2*s1^2                                         0                                         0                                         0                                         0]
[                                        0                                         0                                         0                                         0                                         0                                         0]
[                                        0                                         0                                         0                  1/2*s1^2 - 1/2*s4^2/s1^2                                         0                                         0]
[                                        0                                         0                                         0                                         0             1/2*s4^2/s1^2 - 1/2*s7^2/s4^2                                         0]
[                                        0                                         0                                         0                                         0                                         0                             1/2*s7^2/s4^2]

Now we solve equation $(*)$. Recall that the derivation $D$ can be assumed in diagonal form.

In [30]:
c = var("c")
D = der_gen_diag
display(D)
nilsoliton_eqs = (Ric_on_basis - c * identity_matrix(alg.dimension()) - D).coefficients()
nilsoliton_eqs

[       d0         0         0         0         0         0]
[        0        d2         0         0         0         0]
[        0         0        d5         0         0         0]
[        0         0         0   d0 + d2         0         0]
[        0         0         0         0 2*d0 + d2         0]
[        0         0         0         0         0 3*d0 + d2]

[-1/2*s1^2 - c - d0 - 1/2*s4^2/s1^2 - 1/2*s7^2/s4^2,
 -1/2*s1^2 - c - d2,
 -c - d5,
 1/2*s1^2 - c - d0 - d2 - 1/2*s4^2/s1^2,
 -c - 2*d0 - d2 + 1/2*s4^2/s1^2 - 1/2*s7^2/s4^2,
 -c - 3*d0 - d2 + 1/2*s7^2/s4^2]

In [31]:
g_diag_vars = list(set(Sigma.diagonal()) - {1})
incg = [c] + der_vars_diag + g_diag_vars
display(incg)
%time nilsoliton_sols = solve(nilsoliton_eqs, incg, algorithm='sympy') # long time
for sol in nilsoliton_sols:
    display(sol)

[c, d0, d2, d5, s4, s1, s7]

CPU times: user 15.1 s, sys: 0 ns, total: 15.1 s
Wall time: 15.1 s


{c: -6^(1/3)*(s7^4)^(2/3)/s7^2,
 d0: 1/6*6^(1/3)*(s7^4)^(2/3)/s7^2,
 d2: 3/4*6^(1/3)*(s7^4)^(2/3)/s7^2,
 d5: 6^(1/3)*(s7^4)^(2/3)/s7^2,
 s1: -1/2*3^(1/6)*2^(2/3)*(s7^4)^(1/3)/s7,
 s4: -1/3*3^(5/6)*2^(1/3)*(s7^4)^(1/6)}

{c: -6^(1/3)*(s7^4)^(2/3)/s7^2,
 d0: 1/6*6^(1/3)*(s7^4)^(2/3)/s7^2,
 d2: 3/4*6^(1/3)*(s7^4)^(2/3)/s7^2,
 d5: 6^(1/3)*(s7^4)^(2/3)/s7^2,
 s1: -1/2*3^(1/6)*2^(2/3)*(s7^4)^(1/3)/s7,
 s4: 1/3*3^(5/6)*2^(1/3)*(s7^4)^(1/6)}

{c: -6^(1/3)*(s7^4)^(2/3)/s7^2,
 d0: 1/6*6^(1/3)*(s7^4)^(2/3)/s7^2,
 d2: 3/4*6^(1/3)*(s7^4)^(2/3)/s7^2,
 d5: 6^(1/3)*(s7^4)^(2/3)/s7^2,
 s1: 1/2*3^(1/6)*2^(2/3)*(s7^4)^(1/3)/s7,
 s4: -1/3*3^(5/6)*2^(1/3)*(s7^4)^(1/6)}

...

{c: 1/2*(s7^4)^(2/3)*(-I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 d0: 1/12*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) - 6^(1/3))/s7^2,
 d2: 3/8*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) - 6^(1/3))/s7^2,
 d5: 1/2*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) - 6^(1/3))/s7^2,
 s1: -1/2*2^(1/6)*sqrt(-(s7^4)^(2/3)*(-I*3^(5/6) + 3^(1/3)))/s7,
 s4: 1/6*(s7^4)^(1/6)*(3^(5/6)*2^(1/3) - 3*I*6^(1/3))}

{c: 1/2*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 d0: -1/12*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 d2: -3/8*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 d5: -1/2*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 s1: -1/2*sqrt(-(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3)))/s7,
 s4: -1/6*(s7^4)^(1/6)*(3^(5/6)*2^(1/3) + 3*I*6^(1/3))}

{c: 1/2*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 d0: -1/12*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 d2: -3/8*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 d5: -1/2*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 s1: -1/2*sqrt(-(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3)))/s7,
 s4: 1/6*(s7^4)^(1/6)*(3^(5/6)*2^(1/3) + 3*I*6^(1/3))}

{c: 1/2*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 d0: -1/12*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 d2: -3/8*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 d5: -1/2*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 s1: 1/2*sqrt(-(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3)))/s7,
 s4: -1/6*(s7^4)^(1/6)*(3^(5/6)*2^(1/3) + 3*I*6^(1/3))}

{c: 1/2*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 d0: -1/12*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 d2: -3/8*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 d5: -1/2*(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3))/s7^2,
 s1: 1/2*sqrt(-(s7^4)^(2/3)*(I*3^(5/6)*2^(1/3) + 6^(1/3)))/s7,
 s4: 1/6*(s7^4)^(1/6)*(3^(5/6)*2^(1/3) + 3*I*6^(1/3))}

Since $g = g_\sigma$ with $\sigma \in \Sigma \subset \rm T_6^+$ the only meaningful solution is the fourth one. Warning: the place of the meaningful solution in `nilsoliton_sols` may vary.

In [32]:
g_nil = g_diag.subs(nilsoliton_sols[3]).canonicalize_radical()
Sigma_nil = Sigma_diag.subs(nilsoliton_sols[3]).canonicalize_radical()
ip_nil = inner_product_from_matrix(alg, g_nil)
g_nil

[                           1                            0                            0                            0                            0                            0]
[                           0                            1                            0                            0                            0                            0]
[                           0                            0                            1                            0                            0                            0]
[                           0                            0                            0 1/2*3^(1/3)*2^(1/3)*s7^(2/3)                            0                            0]
[                           0                            0                            0                            0 1/3*3^(2/3)*2^(2/3)*s7^(4/3)                            0]
[                           0                            0                            0                            0                            0                         s7^2]

It follows from Theorem 6.5 that the nilsoliton metrics has nontrivial index of symmetry.

In [33]:
Sigma, Sigma_nil

(
[ 1  0  0  0  0  0]  [                           1                            0                            0                            0                            0                            0]
[ 0  1  0  0  0  0]  [                           0                            1                            0                            0                            0                            0]
[ 0  0  1  0  0  0]  [                           0                            0                            1                            0                            0                            0]
[ 0  0 s0 s1  0  0]  [                           0                            0                            0 1/2*3^(1/6)*2^(2/3)*s7^(1/3)                            0                            0]
[ 0  0 s2 s3 s4  0]  [                           0                            0                            0                            0 1/3*3^(5/6)*2^(1/3)*s7^(2/3)                            0]
[ 0  0  0 s5 s6 s7], [                           0                            0                            0                            0                            0                           s7]
)

### Isometry with the nilsoliton metric given in [Will11]

The nilsoliton metric from the 2011 paper of Will which is isomorphic to $\mathfrak h_{21}$ is the number $12$ given by $(0, 0, 0, 3^\frac12 \, \mathbf{12}, 2 \, \mathbf{14}, 3^\frac12 \, \mathbf{15})$.

In [34]:
d_h = {('x1', 'x2'): {'x4': sqrt(3)}, ('x1', 'x4'): {'x5': 2}, ('x1', 'x5'): {'x6': sqrt(3)}} 
hh_will.<x1,x2,x3,x4,x5,x6> = LieAlgebra(SR, d_h, nilpotent=True) 
for kk in hh_will.structure_coefficients().keys():
    print(kk, " = ", hh_will.structure_coefficients()[kk])

('x1', 'x2')  =  sqrt(3)*x4
('x1', 'x4')  =  2*x5
('x1', 'x5')  =  sqrt(3)*x6


Notice that the basis $x_1, \ldots, x_6$ is orthonormal by definition. We propose the following isomorphism between `hh_will` and `alg`.

In [33]:
e1, e2, e3, e4, e5, e6 = alg.basis()
f1, f2, f3, f4, f5, f6 = e1, e2, e3, -1/sqrt(3) * e4, 1/(2 * sqrt(3)) * e5, -1/6 * e6
hh_will.morphism({x1:f1, x2:f2, x3:f3, x4:f4, x5:f5, x6:f6})

Lie algebra morphism:
  From: Nilpotent Lie algebra on 6 generators (x1, x2, x3, x4, x5, x6) over Symbolic Ring
  To:   Nilpotent Lie algebra on 6 generators (e1, e2, e3, e4, e5, e6) over Symbolic Ring
  Defn: x1 |--> e1
        x2 |--> e2
        x3 |--> e3
        x4 |--> -1/3*sqrt(3)*e4
        x5 |--> 1/6*sqrt(3)*e5
        x6 |--> -1/6*e6

We choose $s_7$ in order to make this isomorphism and isometry. 

In [35]:
on_basis_nil = [f1, f2, f3, f4, f5, f6]
matrix(6, 6, lambda i, j: ip_nil(on_basis_nil[i], on_basis_nil[j])).canonicalize_radical()

[                           1                            0                            0                            0                            0                            0]
[                           0                            1                            0                            0                            0                            0]
[                           0                            0                            1                            0                            0                            0]
[                           0                            0                            0 1/4*3^(1/3)*2^(1/3)*s7^(2/3)                            0                            0]
[                           0                            0                            0                            0 1/3*3^(2/3)*2^(2/3)*s7^(4/3)                            0]
[                           0                            0                            0                            0                            0                     1/4*s7^2]

So it follows that $s_7 = 6$. Indeed,

In [36]:
_.subs(s7=6).canonicalize_radical()

[  1   0   0   0   0   0]
[  0   1   0   0   0   0]
[  0   0   1   0   0   0]
[  0   0   0 3/2   0   0]
[  0   0   0   0  12   0]
[  0   0   0   0   0   9]

So, the nilsoliton metric from Will's paper is

In [37]:
g_will = g_nil.subs(s7=6).canonicalize_radical()
g_will

[ 1  0  0  0  0  0]
[ 0  1  0  0  0  0]
[ 0  0  1  0  0  0]
[ 0  0  0  3  0  0]
[ 0  0  0  0 12  0]
[ 0  0  0  0  0 36]

## Case $\mathfrak h_{22}$

In [38]:
construct_the_algebra("h22")
%time generic_metric(alg_name) # long time
g, Sigma

CPU times: user 3.16 s, sys: 16 ms, total: 3.17 s
Wall time: 3.04 s


(
[                 1                  0                  0                  0                  0                  0]
[                 0               s0^2                  0                  0                  0                  0]
[                 0                  0    s1^2 + s3^2 + 1      s1*s2 + s3*s4              s3*s5                  0]
[                 0                  0      s1*s2 + s3*s4 s2^2 + s4^2 + s6^2      s4*s5 + s6*s7              s6*s8]
[                 0                  0              s3*s5      s4*s5 + s6*s7        s5^2 + s7^2              s7*s8]
[                 0                  0                  0              s6*s8              s7*s8               s8^2],

[ 1  0  0  0  0  0]
[ 0 s0  0  0  0  0]
[ 0  0  1  0  0  0]
[ 0  0 s1 s2  0  0]
[ 0  0 s3 s4 s5  0]
[ 0  0  0 s6 s7 s8]
)

We can avoid very long computations by working with diagonal metrics. 

In [39]:
g_diag = g.subs(s1=0,s3=0,s4=0,s6=0,s7=0)
Sigma_diag = Sigma.subs(s1=0,s3=0,s4=0,s6=0,s7=0)
g_diag

[   1    0    0    0    0    0]
[   0 s0^2    0    0    0    0]
[   0    0    1    0    0    0]
[   0    0    0 s2^2    0    0]
[   0    0    0    0 s5^2    0]
[   0    0    0    0    0 s8^2]

Now we compute the Ricci tensor.

In [40]:
ip = inner_product_from_matrix(alg, g_diag)
on_basis = gram_schmidt(alg, ip)
display(on_basis)
Ric_on_basis = Ric_on(alg, g_diag, on_basis).simplify_full() # we need .simplify_full() otherwise computations take too long
Ric_on_basis

[e1, 1/s0*e2, e3, 1/s2*e4, 1/s5*e5, 1/s8*e6]

[-1/2*(s2^4*s5^2 + s0^2*s5^4 + s0^2*s2^2*s8^2)/(s0^2*s2^2*s5^2)                                                              0                                                              0                                                              0                                                              0                                                              0]
[                                                             0                                 -1/2*(s2^4 + s8^2)/(s0^2*s2^2)                                                              0                                                              0                                                              0                                                              0]
[                                                             0                                                              0                                                              0                                                              0                                                              0                                                              0]
[                                                             0                                                              0                                                              0                      1/2*(s2^4 - s0^2*s5^2 - s8^2)/(s0^2*s2^2)                                                              0                                                              0]
[                                                             0                                                              0                                                              0                                                              0                             1/2*(s5^4 - s2^2*s8^2)/(s2^2*s5^2)                                                              0]
[                                                             0                                                              0                                                              0                                                              0                                                              0                   1/2*(s0^2*s2^2 + s5^2)*s8^2/(s0^2*s2^2*s5^2)]

Now we solve equation $(*)$. Recall that the derivation $D$ can be assumed in diagonal form

In [41]:
c = var("c")
D = der_gen_diag
display(D)
nilsoliton_eqs = (Ric_on_basis - c * identity_matrix(alg.dimension()) - D).coefficients()
nilsoliton_eqs

[  d0    0    0    0    0    0]
[   0 2*d0    0    0    0    0]
[   0    0   d4    0    0    0]
[   0    0    0 3*d0    0    0]
[   0    0    0    0 4*d0    0]
[   0    0    0    0    0 5*d0]

[-c - d0 - 1/2*(s2^4*s5^2 + s0^2*s5^4 + s0^2*s2^2*s8^2)/(s0^2*s2^2*s5^2),
 -c - 2*d0 - 1/2*(s2^4 + s8^2)/(s0^2*s2^2),
 -c - d4,
 -c - 3*d0 + 1/2*(s2^4 - s0^2*s5^2 - s8^2)/(s0^2*s2^2),
 -c - 4*d0 + 1/2*(s5^4 - s2^2*s8^2)/(s2^2*s5^2),
 -c - 5*d0 + 1/2*(s0^2*s2^2 + s5^2)*s8^2/(s0^2*s2^2*s5^2)]

In [42]:
g_diag_vars = list(set(Sigma.diagonal()) - {1})
incg = [c] + der_vars_diag + g_diag_vars
display(incg)
%time nilsoliton_sols = solve(nilsoliton_eqs, incg, algorithm='sympy') # long time
for sol in nilsoliton_sols:
    display(sol)

[c, d0, d4, s5, s8, s0, s2]

CPU times: user 35.5 s, sys: 0 ns, total: 35.5 s
Wall time: 35.5 s


{c: -11/12*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d0: 1/4*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d4: 11/12*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 s0: -1/2*3^(1/8)*2^(7/8)*(s8^6)^(3/8)/s8^2,
 s2: -1/2*3^(1/4)*2^(3/4)*(s8^6)^(1/4)/s8,
 s5: -1/2*3^(3/8)*2^(5/8)*(s8^6)^(1/8)}

{c: -11/12*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d0: 1/4*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d4: 11/12*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 s0: -1/2*3^(1/8)*2^(7/8)*(s8^6)^(3/8)/s8^2,
 s2: -1/2*3^(1/4)*2^(3/4)*(s8^6)^(1/4)/s8,
 s5: 1/2*3^(3/8)*2^(5/8)*(s8^6)^(1/8)}

{c: -11/12*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d0: 1/4*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d4: 11/12*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 s0: -1/2*3^(1/8)*2^(7/8)*(s8^6)^(3/8)/s8^2,
 s2: 1/2*3^(1/4)*2^(3/4)*(s8^6)^(1/4)/s8,
 s5: -1/2*3^(3/8)*2^(5/8)*(s8^6)^(1/8)}

...

{c: 11/12*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d0: -1/4*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d4: -11/12*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 s0: -1/2*3^(1/8)*2^(7/8)*sqrt(-I*(s8^6)^(3/4))/s8^2,
 s2: 1/2*3^(1/4)*2^(3/4)*sqrt(-sqrt(s8^6))/s8,
 s5: (1/2*I + 1/2)*54^(1/8)*(s8^6)^(1/8)}

{c: 11/12*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d0: -1/4*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d4: -11/12*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 s0: 1/2*3^(1/8)*2^(7/8)*sqrt(-I*(s8^6)^(3/4))/s8^2,
 s2: -1/2*3^(1/4)*2^(3/4)*sqrt(-sqrt(s8^6))/s8,
 s5: -(1/2*I + 1/2)*54^(1/8)*(s8^6)^(1/8)}

{c: 11/12*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d0: -1/4*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d4: -11/12*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 s0: 1/2*3^(1/8)*2^(7/8)*sqrt(-I*(s8^6)^(3/4))/s8^2,
 s2: -1/2*3^(1/4)*2^(3/4)*sqrt(-sqrt(s8^6))/s8,
 s5: (1/2*I + 1/2)*54^(1/8)*(s8^6)^(1/8)}

{c: 11/12*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d0: -1/4*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d4: -11/12*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 s0: 1/2*3^(1/8)*2^(7/8)*sqrt(-I*(s8^6)^(3/4))/s8^2,
 s2: 1/2*3^(1/4)*2^(3/4)*sqrt(-sqrt(s8^6))/s8,
 s5: -(1/2*I + 1/2)*54^(1/8)*(s8^6)^(1/8)}

{c: 11/12*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d0: -1/4*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 d4: -11/12*I*3^(1/4)*2^(3/4)*(s8^6)^(3/4)/s8^4,
 s0: 1/2*3^(1/8)*2^(7/8)*sqrt(-I*(s8^6)^(3/4))/s8^2,
 s2: 1/2*3^(1/4)*2^(3/4)*sqrt(-sqrt(s8^6))/s8,
 s5: (1/2*I + 1/2)*54^(1/8)*(s8^6)^(1/8)}

Since $g = g_\sigma$ with $\sigma \in \Sigma \subset \rm T_6^+$ the only meaningful solution is the seventh one. Warning: the place of the meaningful solution in `nilsoliton_sols` may vary.

In [43]:
g_nil = g_diag.subs(nilsoliton_sols[7]).canonicalize_radical()
Sigma_nil = Sigma_diag.subs(nilsoliton_sols[7]).canonicalize_radical()
ip_nil = inner_product_from_matrix(alg, g_nil)
g_nil

[                           1                            0                            0                            0                            0                            0]
[                           0 1/2*3^(1/4)*2^(3/4)*sqrt(s8)                            0                            0                            0                            0]
[                           0                            0                            1                            0                            0                            0]
[                           0                            0                            0       1/2*sqrt(3)*sqrt(2)*s8                            0                            0]
[                           0                            0                            0                            0 1/2*3^(3/4)*2^(1/4)*s8^(3/2)                            0]
[                           0                            0                            0                            0                            0                         s8^2]

It follows from Theorem 6.3 that the nilsoliton metrics has nontrivial index of symmetry.

In [44]:
Sigma, Sigma_nil

(
[ 1  0  0  0  0  0]  [                           1                            0                            0                            0                            0                            0]
[ 0 s0  0  0  0  0]  [                           0 1/2*3^(1/8)*2^(7/8)*s8^(1/4)                            0                            0                            0                            0]
[ 0  0  1  0  0  0]  [                           0                            0                            1                            0                            0                            0]
[ 0  0 s1 s2  0  0]  [                           0                            0                            0 1/2*3^(1/4)*2^(3/4)*sqrt(s8)                            0                            0]
[ 0  0 s3 s4 s5  0]  [                           0                            0                            0                            0 1/2*3^(3/8)*2^(5/8)*s8^(3/4)                            0]
[ 0  0  0 s6 s7 s8], [                           0                            0                            0                            0                            0                           s8]
)

### Isometry with the nilsoliton metric given in [Will11]

The nilsoliton metric from the 2011 paper of Will which is isomorphic to $\mathfrak h_{22}$ is the number $12$ given by $(0, 0, 0, 3^\frac12 \mathbf{12}, 3^\frac12 \mathbf{14}, 2^\frac12\mathbf{15} + 2^\frac12 \mathbf{24})$.

In [45]:
d_h = {('x1', 'x2'): {'x4': sqrt(3)}, ('x1', 'x4'): {'x5': sqrt(3)}, ('x1', 'x5'): {'x6': sqrt(2)}, ('x2', 'x4'): {'x6': sqrt(2)}} 
hh_will.<x1,x2,x3,x4,x5,x6> = LieAlgebra(SR, d_h, nilpotent=True) 
for kk in hh_will.structure_coefficients().keys():
    print(kk, " = ", hh_will.structure_coefficients()[kk])

('x1', 'x2')  =  sqrt(3)*x4
('x1', 'x4')  =  sqrt(3)*x5
('x1', 'x5')  =  sqrt(2)*x6
('x2', 'x4')  =  sqrt(2)*x6


Notice that the basis $x_1, \ldots, x_6$ is orthonormal by definition. We propose the following isomorphism between `hh_will` and `alg`.

In [46]:
e1, e2, e3, e4, e5, e6 = alg.basis()
f1, f2, f3, f4, f5, f6 = e1, -sqrt(3)/3*e2, e3, 1/3*e4, -1/9*sqrt(3)*e5, 1/18*sqrt(3)*sqrt(2)*e6
hh_will.morphism({x1:f1, x2:f2, x3:f3, x4:f4, x5:f5, x6:f6})

Lie algebra morphism:
  From: Nilpotent Lie algebra on 6 generators (x1, x2, x3, x4, x5, x6) over Symbolic Ring
  To:   Nilpotent Lie algebra on 6 generators (e1, e2, e3, e4, e5, e6) over Symbolic Ring
  Defn: x1 |--> e1
        x2 |--> -1/3*sqrt(3)*e2
        x3 |--> e3
        x4 |--> 1/3*e4
        x5 |--> -1/9*sqrt(3)*e5
        x6 |--> 1/18*sqrt(3)*sqrt(2)*e6

We choose $s_8$ in order to make this isomorphism and isometry. 

In [47]:
on_basis_nil = [f1, f2, f3, f4, f5, f6]
matrix(6, 6, lambda i, j: ip_nil(on_basis_nil[i], on_basis_nil[j])).canonicalize_radical()

[                            1                             0                             0                             0                             0                             0]
[                            0  1/6*3^(1/4)*2^(3/4)*sqrt(s8)                             0                             0                             0                             0]
[                            0                             0                             1                             0                             0                             0]
[                            0                             0                             0       1/18*sqrt(3)*sqrt(2)*s8                             0                             0]
[                            0                             0                             0                             0 1/54*3^(3/4)*2^(1/4)*s8^(3/2)                             0]
[                            0                             0                             0                             0                             0                     1/54*s8^2]

So it follows that $s_8 = \sqrt{54} = 3 \sqrt 6$. Indeed,

In [48]:
_.subs(s8=3*sqrt(6)).canonicalize_radical()

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]

So, the nilsoliton metric from Will's paper is

In [49]:
g_will = g_nil.subs(s8=3*sqrt(6)).canonicalize_radical()
g_will

[ 1  0  0  0  0  0]
[ 0  3  0  0  0  0]
[ 0  0  1  0  0  0]
[ 0  0  0  9  0  0]
[ 0  0  0  0 27  0]
[ 0  0  0  0  0 54]

## Case $\mathfrak h_{28}$

In [50]:
construct_the_algebra("h28")
%time generic_metric(alg_name) # long time
g, Sigma

CPU times: user 4.14 s, sys: 79 ms, total: 4.22 s
Wall time: 3.63 s


(
[                        1                         0                         0                         0                         0                         0]
[                        0                         1                         0                         0                         0                         0]
[                        0                         0 s0^2 + s1^2 + s3^2 + s6^2     s1*s2 + s3*s4 + s6*s7             s3*s5 + s6*s8                     s6*s9]
[                        0                         0     s1*s2 + s3*s4 + s6*s7        s2^2 + s4^2 + s7^2             s4*s5 + s7*s8                     s7*s9]
[                        0                         0             s3*s5 + s6*s8             s4*s5 + s7*s8               s5^2 + s8^2                     s8*s9]
[                        0                         0                     s6*s9                     s7*s9                     s8*s9                      s9^2],

[ 1  0  0  0  0  0]
[ 0  1  0  0  0  0]
[ 0  0 s0  0  0  0]
[ 0  0 s1 s2  0  0]
[ 0  0 s3 s4 s5  0]
[ 0  0 s6 s7 s8 s9]
)

We can avoid very long computations by working with diagonal metrics. 

In [51]:
g_diag = g.subs(s1=0,s3=0,s4=0,s6=0,s7=0,s8=0)
Sigma_diag = Sigma.subs(s1=0,s3=0,s4=0,s6=0,s7=0,s8=0)
g_diag

[   1    0    0    0    0    0]
[   0    1    0    0    0    0]
[   0    0 s0^2    0    0    0]
[   0    0    0 s2^2    0    0]
[   0    0    0    0 s5^2    0]
[   0    0    0    0    0 s9^2]

Now we compute the Ricci tensor.

In [52]:
ip = inner_product_from_matrix(alg, g_diag)
on_basis = gram_schmidt(alg, ip)
display(on_basis)
Ric_on_basis = Ric_on(alg, g_diag, on_basis).simplify_full() # we need .simplify_full() otherwise computations take too long
Ric_on_basis

[e1, e2, 1/s0*e3, 1/s2*e4, 1/s5*e5, 1/s9*e6]

[-1/2*(s0^2*s5^4 + s0^2*s2^2*s9^2 + (s0^4*s2^2 + s2^4)*s5^2)/(s0^2*s2^2*s5^2)                                                                            0                                                                            0                                                                            0                                                                            0                                                                            0]
[                                                                           0                                                                    -1/2*s0^2                                                                            0                                                                            0                                                                            0                                                                            0]
[                                                                           0                                                                            0                                                       1/2*(s0^4 - s2^2)/s0^2                                                                            0                                                                            0                                                                            0]
[                                                                           0                                                                            0                                                                            0                                           1/2*(s2^4 - s0^2*s5^2)/(s0^2*s2^2)                                                                            0                                                                            0]
[                                                                           0                                                                            0                                                                            0                                                                            0                                           1/2*(s5^4 - s2^2*s9^2)/(s2^2*s5^2)                                                                            0]
[                                                                           0                                                                            0                                                                            0                                                                            0                                                                            0                                                                1/2*s9^2/s5^2]

Now we solve equation $(*)$. Recall that the derivation $D$ can be assumed in diagonal form

In [53]:
c = var("c")
D = der_gen_diag
display(D)
nilsoliton_eqs = (Ric_on_basis - c * identity_matrix(alg.dimension()) - D).coefficients()
nilsoliton_eqs

[       d0         0         0         0         0         0]
[        0        d2         0         0         0         0]
[        0         0   d0 + d2         0         0         0]
[        0         0         0 2*d0 + d2         0         0]
[        0         0         0         0 3*d0 + d2         0]
[        0         0         0         0         0 4*d0 + d2]

[-c - d0 - 1/2*(s0^2*s5^4 + s0^2*s2^2*s9^2 + (s0^4*s2^2 + s2^4)*s5^2)/(s0^2*s2^2*s5^2),
 -1/2*s0^2 - c - d2,
 -c - d0 - d2 + 1/2*(s0^4 - s2^2)/s0^2,
 -c - 2*d0 - d2 + 1/2*(s2^4 - s0^2*s5^2)/(s0^2*s2^2),
 -c - 3*d0 - d2 + 1/2*(s5^4 - s2^2*s9^2)/(s2^2*s5^2),
 -c - 4*d0 - d2 + 1/2*s9^2/s5^2]

In [54]:
g_diag_vars = list(set(Sigma.diagonal()) - {1})
incg = [c] + der_vars_diag + g_diag_vars
display(incg)
%time nilsoliton_sols = solve(nilsoliton_eqs, incg, algorithm='sympy') # long time
for sol in nilsoliton_sols:
    display(sol)

[c, d0, d2, s5, s0, s9, s2]

CPU times: user 33.4 s, sys: 63 ms, total: 33.5 s
Wall time: 33.6 s


{c: -11/12*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d0: 1/12*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d2: 3/4*sqrt(6)*(s9^6)^(3/4)/s9^4,
 s0: -1/3*3^(3/4)*2^(1/4)*(s9^6)^(3/8)/s9^2,
 s2: -(s9^6)^(1/4)/s9,
 s5: -1/2*3^(1/4)*2^(3/4)*(s9^6)^(1/8)}

{c: -11/12*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d0: 1/12*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d2: 3/4*sqrt(6)*(s9^6)^(3/4)/s9^4,
 s0: -1/3*3^(3/4)*2^(1/4)*(s9^6)^(3/8)/s9^2,
 s2: -(s9^6)^(1/4)/s9,
 s5: 1/2*3^(1/4)*2^(3/4)*(s9^6)^(1/8)}

{c: -11/12*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d0: 1/12*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d2: 3/4*sqrt(6)*(s9^6)^(3/4)/s9^4,
 s0: -1/3*3^(3/4)*2^(1/4)*(s9^6)^(3/8)/s9^2,
 s2: (s9^6)^(1/4)/s9,
 s5: -1/2*3^(1/4)*2^(3/4)*(s9^6)^(1/8)}

...

{c: 11/12*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d0: -1/12*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d2: -3/4*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 s0: -1/3*3^(3/4)*2^(1/4)*sqrt(-I*(s9^6)^(3/4))/s9^2,
 s2: sqrt(-sqrt(s9^6))/s9,
 s5: (1/2*I + 1/2)*6^(1/4)*(s9^6)^(1/8)}

{c: 11/12*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d0: -1/12*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d2: -3/4*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 s0: 1/3*3^(3/4)*2^(1/4)*sqrt(-I*(s9^6)^(3/4))/s9^2,
 s2: -sqrt(-sqrt(s9^6))/s9,
 s5: -(1/2*I + 1/2)*6^(1/4)*(s9^6)^(1/8)}

{c: 11/12*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d0: -1/12*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d2: -3/4*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 s0: 1/3*3^(3/4)*2^(1/4)*sqrt(-I*(s9^6)^(3/4))/s9^2,
 s2: -sqrt(-sqrt(s9^6))/s9,
 s5: (1/2*I + 1/2)*6^(1/4)*(s9^6)^(1/8)}

{c: 11/12*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d0: -1/12*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d2: -3/4*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 s0: 1/3*3^(3/4)*2^(1/4)*sqrt(-I*(s9^6)^(3/4))/s9^2,
 s2: sqrt(-sqrt(s9^6))/s9,
 s5: -(1/2*I + 1/2)*6^(1/4)*(s9^6)^(1/8)}

{c: 11/12*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d0: -1/12*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 d2: -3/4*I*sqrt(6)*(s9^6)^(3/4)/s9^4,
 s0: 1/3*3^(3/4)*2^(1/4)*sqrt(-I*(s9^6)^(3/4))/s9^2,
 s2: sqrt(-sqrt(s9^6))/s9,
 s5: (1/2*I + 1/2)*6^(1/4)*(s9^6)^(1/8)}

Since $g = g_\sigma$ with $\sigma \in \Sigma \subset \rm T_6^+$ the only meaningful solution is the seventh one. Warning: the place of the meaningful solution in `nilsoliton_sols` may vary.

In [55]:
g_nil = g_diag.subs(nilsoliton_sols[7]).canonicalize_radical()
Sigma_nil = Sigma_diag.subs(nilsoliton_sols[7]).canonicalize_radical()
ip_nil = inner_product_from_matrix(alg, g_nil)
g_nil

[                           1                            0                            0                            0                            0                            0]
[                           0                            1                            0                            0                            0                            0]
[                           0                            0 1/3*sqrt(3)*sqrt(2)*sqrt(s9)                            0                            0                            0]
[                           0                            0                            0                           s9                            0                            0]
[                           0                            0                            0                            0 1/2*sqrt(3)*sqrt(2)*s9^(3/2)                            0]
[                           0                            0                            0                            0                            0                         s9^2]

It follows from Theorem 6.7 that the nilsoliton metrics has nontrivial index of symmetry.

In [56]:
Sigma, Sigma_nil

(
[ 1  0  0  0  0  0]  [                           1                            0                            0                            0                            0                            0]
[ 0  1  0  0  0  0]  [                           0                            1                            0                            0                            0                            0]
[ 0  0 s0  0  0  0]  [                           0                            0 1/3*3^(3/4)*2^(1/4)*s9^(1/4)                            0                            0                            0]
[ 0  0 s1 s2  0  0]  [                           0                            0                            0                     sqrt(s9)                            0                            0]
[ 0  0 s3 s4 s5  0]  [                           0                            0                            0                            0 1/2*3^(1/4)*2^(3/4)*s9^(3/4)                            0]
[ 0  0 s6 s7 s8 s9], [                           0                            0                            0                            0                            0                           s9]
)

### Isometry with the nilsoliton metric given in [Will11]

The nilsoliton metric from the 2011 paper of Will which is isomorphic to $\mathfrak h_{28}$ is the number $3$ given by $(0, 0, 2 \, \mathbf{12}, 6^\frac12 \, \mathbf{13}, 6^\frac12 \, \mathbf{14}, 2 \, \mathbf{15})$.

In [57]:
d_h = {('x1', 'x2'): {'x3': 2}, ('x1', 'x3'): {'x4': sqrt(6)}, ('x1', 'x4'): {'x5': sqrt(6)}, ('x1', 'x5'): {'x6': 2}} 
hh_will.<x1,x2,x3,x4,x5,x6> = LieAlgebra(SR, d_h, nilpotent=True) 
for kk in hh_will.structure_coefficients().keys():
    print(kk, " = ", hh_will.structure_coefficients()[kk])

('x1', 'x2')  =  2*x3
('x1', 'x3')  =  sqrt(6)*x4
('x1', 'x4')  =  sqrt(6)*x5
('x1', 'x5')  =  2*x6


Notice that the basis $x_1, \ldots, x_6$ is orthonormal by definition. We propose the following isomorphism between `hh_will` and `alg`.

In [58]:
e1, e2, e3, e4, e5, e6 = alg.basis()
f1, f2, f3, f4, f5, f6 = e1, e2, -1/2 * e3, 1/(2*sqrt(6)) * e4, -1/12 * e5, 1/24 * e6
hh_will.morphism({x1:f1, x2:f2, x3:f3, x4:f4, x5:f5, x6:f6})

Lie algebra morphism:
  From: Nilpotent Lie algebra on 6 generators (x1, x2, x3, x4, x5, x6) over Symbolic Ring
  To:   Nilpotent Lie algebra on 6 generators (e1, e2, e3, e4, e5, e6) over Symbolic Ring
  Defn: x1 |--> e1
        x2 |--> e2
        x3 |--> -1/2*e3
        x4 |--> 1/12*sqrt(6)*e4
        x5 |--> -1/12*e5
        x6 |--> 1/24*e6

We choose $s_9$ in order to make this isomorphism and isometry. 

In [59]:
on_basis_nil = [f1, f2, f3, f4, f5, f6]
matrix(6, 6, lambda i, j: ip_nil(on_basis_nil[i], on_basis_nil[j])).canonicalize_radical()

[                             1                              0                              0                              0                              0                              0]
[                             0                              1                              0                              0                              0                              0]
[                             0                              0  1/12*sqrt(3)*sqrt(2)*sqrt(s9)                              0                              0                              0]
[                             0                              0                              0                        1/24*s9                              0                              0]
[                             0                              0                              0                              0 1/288*sqrt(3)*sqrt(2)*s9^(3/2)                              0]
[                             0                              0                              0                              0                              0                     1/576*s9^2]

So it follows that $s_9 = 24$. Indeed,

In [60]:
_.subs(s9=24).canonicalize_radical()

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]

So, the nilsoliton metric from Will's paper is

In [61]:
g_will = g_nil.subs(s9=24).canonicalize_radical()
g_will

[  1   0   0   0   0   0]
[  0   1   0   0   0   0]
[  0   0   4   0   0   0]
[  0   0   0  24   0   0]
[  0   0   0   0 144   0]
[  0   0   0   0   0 576]

---

This notebook corresponds to the article "The moduli space of left-invariant metrics on six-dimensional characteristically solvable nilmanifolds" by I. Cardoso, A. Cosgaya, and S. Reggiani (2024).